# 混淆矩阵
#### 适用于不平衡的数据分类问题
行代表真实值, 列代表预测值

|           |    0 | 1  |
| :-------- | --------:| :--: |
| 0  | TN | FP  |
| 1  | FN |  TP |

1 一般是我们关注的事件

### 精准率 $precision = \frac{TP}{TP+FP}$ 预测10次,这10次正确的概率
### 召回率 $recall = \frac{TP}{TP+FN}$ 关注的事件发生了, 我们成功预测了多少

---------

# 实现混淆矩阵\精准率\召回率

In [4]:
import numpy as np
from sklearn import datasets

In [5]:
digits = datasets.load_digits()
X = digits.data
y = digits.target.copy()

In [7]:
# 手动构造偏斜skew数据
y[digits.target==9] = 1
y[digits.target!=9] = 0
y[y!=1].shape

(1617,)

In [8]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train,y_test = train_test_split(X,y, random_state=666)

In [9]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

0.9755555555555555

In [10]:
y_log_predict = log_reg.predict(X_test)

In [11]:
def TN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true==0) & (y_predict==0))

TN(y_test, y_log_predict)

403

In [12]:
def FP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true==0) & (y_predict==1))

FP(y_test, y_log_predict)

2

In [13]:
def FN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true==1) & (y_predict==0))

FN(y_test, y_log_predict)

9

In [14]:
def TP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true==1) & (y_predict==1))

TP(y_test, y_log_predict)

36

In [15]:
def confusion_matrix(y_true, y_predict):
    return np.array([[TN(y_true, y_predict),FP(y_true, y_predict)],
                    [FN(y_true, y_predict),TP(y_true, y_predict)]])

confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]])

In [16]:
def precision_score(y_true, y_predict):
    tp = TP(y_true, y_predict)
    fp = FP(y_true, y_predict)
    try:
        return tp/(tp+fp)
    except:
        return 0.0

precision_score(y_test, y_log_predict)

0.9473684210526315

In [17]:
def recall_score(y_true, y_predict):
    tp = TP(y_true, y_predict)
    fn = FN(y_true, y_predict)
    try:
        return tp/(tp+fn)
    except:
        return 0.0

recall_score(y_test, y_log_predict)

0.8

# Scikit-learn中的混淆矩阵, 精准率, 召回率

In [18]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]], dtype=int64)

In [19]:
from sklearn.metrics import precision_score

precision_score(y_test, y_log_predict)

0.9473684210526315

In [20]:
from sklearn.metrics import recall_score

recall_score(y_test, y_log_predict)

0.8

# F1 Score兼顾精准率和召回率
F1 Score是precision和recall的调和平均值
$\frac{1}{F1}=\frac{1}{2}(\frac{1}{precision}+\frac{1}{recall})$
## $F1 = \frac{2\cdot precision\cdot recall}{precision+recall}$


In [21]:
import numpy as np

def f1_score(precision, recall):
    try:
        return 2*precision*recall/(precision+recall)
    except:
        return 0.0

In [22]:
precision =0.5
recall = 0.5
f1_score(precision, recall)

0.5

In [23]:
precision =0.1
recall = 0.9
f1_score(precision, recall)

0.18000000000000002

In [24]:
import numpy as np
from sklearn import datasets

digits = datasets.load_digits()
X = digits.data
y = digits.target.copy()

y[digits.target==9] = 1
y[digits.target!=9] = 0

from sklearn.model_selection import train_test_split

X_train,X_test, y_train,y_test = train_test_split(X,y, random_state=666)

In [25]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

0.9755555555555555

In [26]:
y_predict = log_reg.predict(X_test)

In [27]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_predict)

array([[403,   2],
       [  9,  36]], dtype=int64)

In [28]:
from sklearn.metrics import precision_score,recall_score
print(precision_score(y_test, y_predict))
print(recall_score(y_test, y_predict))

0.9473684210526315
0.8


In [29]:
from sklearn.metrics import f1_score

f1_score(y_test, y_predict)

0.8674698795180723

# decision_function
修改决策边界的 threshold 改变precision和recall

In [31]:
decision_scores = log_reg.decision_function(X_test)
decision_scores

array([-22.05698737, -33.02937619, -16.21332482, -80.37914497,
       -48.25127218, -24.540052  , -44.3917185 , -25.04291075,
        -0.9782965 , -19.71744559, -66.25140542, -51.095976  ,
       -31.49346952, -46.05338301, -38.67870918, -29.80471224,
       -37.58846002, -82.57567291, -37.81904399, -11.01163509,
        -9.17440656, -85.13002979, -16.71617095, -46.23719399,
        -5.32989793, -47.91764215, -11.66729562, -39.1959603 ,
       -25.25292315, -14.36646664, -16.99782425, -28.91903142,
       -34.33938565, -29.47599298,  -7.85811474,  -3.82096582,
       -24.0815564 , -22.16363196, -33.61212604, -23.14019304,
       -26.91801894, -62.38934517, -38.85684738, -66.77256306,
       -20.14479378, -17.47885338, -18.06798544, -22.22223522,
       -29.6230231 , -19.73173031,   1.49553174,   8.3207997 ,
       -36.29301444, -42.50729754, -25.90460604, -34.9895844 ,
        -8.42008139, -50.04726697, -51.48206437,  19.88957006,
        -8.91885763, -31.99340597, -11.66095184,  -0.47

In [32]:
decision_scores.max()

19.889570056817906

In [33]:
decision_scores.min()

-85.68605241251701

In [34]:
# 这里修改 decision_scores 的阈值, 改变精准率和召回率
y_predict_2 = np.array(decision_scores >= 5, dtype='int')

In [35]:
confusion_matrix(y_test, y_predict_2)

array([[404,   1],
       [ 21,  24]], dtype=int64)

In [36]:
precision_score(y_test, y_predict_2)

0.96

In [37]:
recall_score(y_test, y_predict_2)

0.5333333333333333

In [38]:
f1_score(y_test, y_predict_2)

0.6857142857142858